In [2]:
import pandas as pd
import sys,os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


df_train=pd.read_csv('Data/train.csv')
df_test=pd.read_csv('Data/test.csv')
df_chal=pd.read_csv('Data/challenge_data.csv')
df_chal['publish_date']=pd.to_datetime(df_chal['publish_date'],format='%d-%m-%Y')
df_chal.sort_values(by='publish_date',inplace=True)

print ("Done")
#df_chal['challenge_id']=np.arange(len(df_chal))

Done


In [24]:



df_chal['challenge_id']=np.arange(len(df_chal))

users=df_train[['user_id']].drop_duplicates()
np.random.seed(1234)
users['dtype']=np.where(np.random.rand(len(users))>0.75,'valid','train')
users=users.set_index('user_id')['dtype']
df_train['ds']=df_train['user_id'].map(users)
df_test['ds']='test'
master=pd.merge(pd.concat([df_train,df_test]),df_chal,left_on=['challenge'],right_on=['challenge_ID'])
master.sort_values(by=['user_id','challenge_sequence'],inplace=True)

def get_datasegment(master,valid=True):
    if valid==True:
        a1=master[master['ds'].isin(['train'])]
        a2=master[master['ds'].isin(['valid'])]
        a3=a2[a2['challenge_sequence']<=10]
        a4=a2[a2['challenge_sequence']>10]
        return a1,a3,a4
    else:
        return master[master['ds'].isin(['train','valid'])],master[master['ds'].isin(['test'])],[]

train,test,target=get_datasegment(master,valid=False)

a1=pd.concat([train,test])
a1['sample']=a1['user_id']%10

freq_items=[]
group=a1[a1['sample']==1][['user_id','challenge_id']]
for name,group in a1.groupby('sample'):
    a2=pd.merge(group,group,on=['user_id'])
    summ=a2.groupby(['challenge_id_x','challenge_id_y'])['user_id'].count().reset_index()
    freq_items.append(summ)
    
freq_items=pd.concat(freq_items).groupby(['challenge_id_x','challenge_id_y'])['user_id'].sum()
freq_items=freq_items.sort_values(ascending=False).to_frame('count_comb').reset_index()
freq_items=freq_items[freq_items['count_comb']>25]

predicted_challenges={}
for name,group in test[['user_id','challenge_id','challenge_sequence']].groupby(['user_id']):
    t1=pd.merge(group,freq_items,left_on=['challenge_id'],right_on=['challenge_id_x'])
    t1=t1[~t1['challenge_id_y'].isin(group['challenge_id'])]
    predicted_challenges[name]=t1.groupby('challenge_id_y')['count_comb'].sum().sort_values(ascending=False).head(10).index.tolist()
    
output=[]
for key,value in predicted_challenges.items():
    output.append([str(int(key))+'_11',value[0] if len(value)>=1 else 1])
    output.append([str(int(key))+'_12',value[1] if len(value)>=2 else 2])
    output.append([str(int(key))+'_13',value[2] if len(value)>=3 else 3])
    output.append([str(int(key))+'_14',value[3] if len(value)>=4 else 4])
    output.append([str(int(key))+'_15',value[4] if len(value)>=5 else 5])
    output.append([str(int(key))+'_16',value[5] if len(value)>=6 else 6])
    output.append([str(int(key))+'_17',value[6] if len(value)>=7 else 7])
    output.append([str(int(key))+'_18',value[7] if len(value)>=8 else 8])
    output.append([str(int(key))+'_19',value[8] if len(value)>=9 else 9])
    output.append([str(int(key))+'_20',value[9] if len(value)>=10 else 10])
predicted_challenges_df=pd.DataFrame(output,columns=['user_sequence','challenge_id'])
predicted_challenges_df=pd.merge(predicted_challenges_df,df_chal[['challenge_id','challenge_ID']],on='challenge_id')
predicted_challenges_df['challenge']=predicted_challenges_df['challenge_ID']
#predicted_challenges_df[['user_sequence','challenge']].to_csv('03.Submissions/01.FreqItemsets2.csv',index=False)    

In [29]:
output=[]
for key,value in predicted_challenges.items():
    output.append([str(int(key))+'_11',value[0] if len(value)>=1 else 1])
    output.append([str(int(key))+'_12',value[1] if len(value)>=2 else 2])
    output.append([str(int(key))+'_13',value[2] if len(value)>=3 else 3])
    output.append([str(int(key))+'_14',value[3] if len(value)>=4 else 4])
    output.append([str(int(key))+'_15',value[4] if len(value)>=5 else 5])
    output.append([str(int(key))+'_16',value[5] if len(value)>=6 else 6])
    output.append([str(int(key))+'_17',value[6] if len(value)>=7 else 7])
    output.append([str(int(key))+'_18',value[7] if len(value)>=8 else 8])
    output.append([str(int(key))+'_19',value[8] if len(value)>=9 else 9])
    output.append([str(int(key))+'_20',value[9] if len(value)>=10 else 10])
predicted_challenges_df=pd.DataFrame(output,columns=['user_sequence','challenge_id'])
predicted_challenges_df=pd.merge(predicted_challenges_df,df_chal[['challenge_id','challenge_ID']],on='challenge_id')
predicted_challenges_df['challenge']=predicted_challenges_df['challenge_ID']

In [33]:
output_a = predicted_challenges_df

In [25]:

df_test['value'] = 1
users_items_pivot_matrix_df = df_test.pivot(index='user_id', 
                                                          columns='challenge', 
                                                          values='value').fillna(0)



from scipy.sparse.linalg import svds
NUMBER_OF_FACTORS_MF = 100
users_items_pivot_matrix = users_items_pivot_matrix_df.as_matrix()
users_ids = list(users_items_pivot_matrix_df.index)
#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)

sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
cf_preds_df = pd.DataFrame(all_user_predicted_ratings, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()

print ("SVD Done")
cf_pred_value = cf_preds_df.transpose()

output_Nor ={}
for i in cf_pred_value.index:
    v = cf_pred_value[cf_pred_value.index==i]
    v_t = v.transpose()
    v_t1 = v_t.sort_values([i],ascending=False)
    list_df = list(df_test[df_test.user_id==i]['challenge'])
    count = 1
    for j in v_t1.index:
        if j not in list_df:
            aa = str(i)+"_"+str(count)
            output_Nor[aa] = j
            count=count+1
        if count>=10:
            break
print ("Done")




SVD Done
Done


In [31]:
output21 =   pd.DataFrame(output_Nor, index=[1])
a1 = output21.transpose()
output_s = a1.reset_index()
output_s.columns = ['user_sequence','challenge']

In [58]:
output_s['user'] = output_s.user_sequence.str.split("_",1).str[0]
output_s['sequence'] = output_s.user_sequence.str.split("_",1).str[1]
output_s[['user','sequence']] = output_s[['user','sequence']].apply(pd.to_numeric)
output_s.head()

,user_sequence,challenge,user,sequence
0,100001_1,CI26917,100001,1
1,100001_2,CI26922,100001,2
2,100001_3,CI26923,100001,3
3,100001_4,CI26905,100001,4
4,100001_5,CI26925,100001,5


In [60]:
output_a['user'] = output_a.user_sequence.str.split("_",1).str[0]
output_a['sequence'] = output_a.user_sequence.str.split("_",1).str[1]
output_a[['user','sequence']] = output_a[['user','sequence']].apply(pd.to_numeric)
output_a.head()

,user_sequence,challenge_id,challenge_ID,challenge,user,sequence
0,4577_11,769,CI23691,CI23691,4577,11
1,4578_13,769,CI23691,CI23691,4578,13
2,4583_13,769,CI23691,CI23691,4583,13
3,4584_15,769,CI23691,CI23691,4584,15
4,4586_14,769,CI23691,CI23691,4586,14


In [65]:
output_s['score'] = 1/output_s['sequence']

In [62]:
output_aa = output_a[['user_sequence','challenge','user','sequence']]

In [64]:
output_aa['sequence'] = output_aa['sequence']-10
output_aa['score'] = 1/output_aa['sequence']
output_aa.head()

C:\Users\shubham.gupta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\shubham.gupta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,user_sequence,challenge,user,sequence,score
0,4577_11,CI23691,4577,1,1.000000
1,4578_13,CI23691,4578,3,0.333333
2,4583_13,CI23691,4583,3,0.333333
3,4584_15,CI23691,4584,5,0.200000
4,4586_14,CI23691,4586,4,0.250000


In [66]:
output_ss = output_s.append(output_aa,ignore_index=False)
output_ss.head()

,user_sequence,challenge,user,sequence,score
0,100001_1,CI26917,100001,1,1.000000
1,100001_2,CI26922,100001,2,0.500000
2,100001_3,CI26923,100001,3,0.333333
3,100001_4,CI26905,100001,4,0.250000
4,100001_5,CI26925,100001,5,0.200000


In [75]:
results = output_ss.groupby(['user','challenge'])['score'].sum().sort_values(ascending=False).reset_index()
results.head()

,user,challenge,score
0,31751,CI26910,2.0
1,112513,CI26917,2.0
2,22854,CI28581,2.0
3,15385,CI27051,2.0
4,35272,CI26876,2.0


In [82]:
results_s = results[results.score>0.3]

In [ ]:
results_s



In [ ]:
results_s['Score_Rank'] = results_s.groupby('user')['score'].rank(ascending=False)
results_s.head()

In [74]:
output_ss['Rank']["score"].rank(ascending=False)
output_ss.head()

ValueError: 

In [70]:
output21 =   pd.DataFrame(output, index=[1])
a1 = output21.transpose()
a1 = a1.reset_index()
a1.columns = ['user_sequence','challenge']
dir_1 = 'C:/Users/shubham.gupta/OneDrive - Opera Solutions USA, LLC/Shubham/Analytics Vidhya/McKinsey_Recommendation/submission/'
file_name = dir_1+ "Freq_2.csv"
a1.to_csv(file_name,index=False)

In [37]:
output1 = pd.DataFrame(output.items())
output1.to_csv('model1.csv',index=False)

In [20]:
train_1  = pd.get_dummies(train, columns=["programming_language", "challenge_series_ID","author_gender","category_id"], prefix=["PL", "CL","AG","CID"])

In [34]:
output

,100001_11,100001_12,100001_13,100003_11,100003_12,100003_13,100005_11,100005_12,100005_13,100007_11,...,99992_13,99994_11,99994_12,99994_13,99997_11,99997_12,99997_13,99999_11,99999_12,99999_13
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
train_1.head()

,user_sequence,user_id,challenge_sequence,challenge,ds,challenge_ID,total_submissions,publish_date,author_ID,author_org_ID,...,CID_230.0,CID_233.0,CID_238.0,CID_239.0,CID_246.0,CID_247.0,CID_250.0,CID_259.0,CID_274.0,CID_304.0
40015,4580_1,4580,1,CI23663,train,CI23663,8897.0,2004-09-15,AI563724,AOI100098,...,0,0,0,0,0,0,0,0,0,0
12459,4580_2,4580,2,CI23855,train,CI23855,20993.0,2006-07-24,AI563823,NaN,...,0,0,0,0,0,0,0,0,0,0
52092,4580_3,4580,3,CI23933,train,CI23933,15086.0,2006-07-27,AI563783,AOI100149,...,0,0,0,0,0,0,0,0,0,0
73634,4580_4,4580,4,CI23975,train,CI23975,9204.0,2005-01-24,AI563823,NaN,...,0,0,0,0,0,0,0,0,0,0
127564,4580_5,4580,5,CI24530,train,CI24530,6814.0,2006-09-26,AI563815,AOI100173,...,0,0,0,0,0,0,0,0,0,0


In [22]:
col = list(train_1.columns)
remove_list = ['user_sequence','user_id','challenge_sequence','challenge','ds','challenge_ID','total_submissions','publish_date','author_ID','author_org_ID']
col = list(filter(lambda x: x not in remove_list, col))

train_grp = train_1.groupby(['user_id'])[col].sum()

In [26]:
train_grp['user_id'] = train_grp.index
train_grp.to_csv('train_pre_processing.csv',index=False)

In [27]:
test_1  = pd.get_dummies(test, columns=["programming_language", "challenge_series_ID","author_gender","category_id"], prefix=["PL", "CL","AG","CID"])
col = list(test_1.columns)
remove_list = ['user_sequence','user_id','challenge_sequence','challenge','ds','challenge_ID','total_submissions','publish_date','author_ID','author_org_ID']
col = list(filter(lambda x: x not in remove_list, col))

test_grp = test_1.groupby(['user_id'])[col].sum()

In [33]:
columns_test = test_grp.columns
for i in columns_test:
    test_grp[i] = (test_grp[i]*1.3)
test_grp = test_grp.reset_index()
test_grp.to_csv('test_pre_processing.csv',index=False)


In [58]:
a = list(train_grp.columns)
b = list(test_grp.columns)
for i in a:
    if i not in b:
        test_grp[i] = 0
        
        
a = list(train_grp.columns)
b = list(test_grp.columns)
for i in b:
    if i not in a:
        test_grp.drop(i, axis = 1, inplace = True)

,user_id,PL_1,PL_2,PL_3,CL_SI2435,CL_SI2436,CL_SI2439,CL_SI2440,CL_SI2441,CL_SI2442,...,CID_219.0,CID_220.0,CID_221.0,CID_223.0,CID_224.0,CID_225.0,CID_226.0,CID_238.0,CID_247.0,CID_259.0
1,4582,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [114]:
test_vk = test_grp.values.tolist()
train_vk = train_grp.values.tolist()

In [ ]:
test_vk_sample = test_vk
final_results = {}
for vk in test_vk:
    v1 = vk[1:]
    corr_value  = {}
    for vtrain in train_vk:
        v2 = vtrain[:-1]
        corr_v = np.corrcoef(v1,v2)
        corr_value[vtrain[-1]] = corr_v[0][1]
    
    v1_list=[]
    newA = list(sorted(corr_value.items(), key=operator.itemgetter(1), reverse=True)[:5])
    for j in newA:
        v1_list.append(j[0])
    final_results[vk[0]] = v1_list
print ("Done")

In [139]:
a = pd.DataFrame(final_results)
a.head()

,4576.0,4582.0
0,62046,62046
1,99656,67943
2,110260,84852
3,67943,99656
4,70860,71679


In [122]:
corr_value_s = sorted(corr_value.values())

In [128]:
newA = list(sorted(corr_value.items(), key=operator.itemgetter(1), reverse=True)[:5])
newA

[(62046, 0.5658685920389882),
 (99656, 0.5646864285075788),
 (110260, 0.5637739716181243),
 (67943, 0.5634236807449161),
 (70860, 0.5616045206342356)]

(62046, 0.5658685920389882)

In [109]:
test_grp.columns

Index(['user_id', 'PL_1', 'PL_2', 'PL_3', 'CL_SI2435', 'CL_SI2436',
       'CL_SI2439', 'CL_SI2440', 'CL_SI2441', 'CL_SI2442',
       ...
       'CID_164.0', 'CID_207.0', 'CID_208.0', 'CID_230.0', 'CID_233.0',
       'CID_239.0', 'CID_246.0', 'CID_250.0', 'CID_274.0', 'CID_304.0'],
      dtype='object', length=620)

In [110]:
train_grp.columns

Index(['PL_1', 'PL_2', 'PL_3', 'CL_SI2435', 'CL_SI2436', 'CL_SI2437',
       'CL_SI2438', 'CL_SI2439', 'CL_SI2440', 'CL_SI2441',
       ...
       'CID_233.0', 'CID_238.0', 'CID_239.0', 'CID_246.0', 'CID_247.0',
       'CID_250.0', 'CID_259.0', 'CID_274.0', 'CID_304.0', 'user_id'],
      dtype='object', length=620)